In [2]:
from sentence_transformers import SentenceTransformer
import os
import json
import pickle
import torch
import os
import json
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
root='/media/jh/新加卷1'
with open(f'{root}/2024_11_10/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json') as f:
    gnd_jsons=json.load(f)
labels=[]
for c in gnd_jsons:
    item1={}
    item1['sentence2']='Classification Name is '+c['Classification Name']+'. Name is '+c['Name']
    item1['Code']=c['Code']
    labels.append(item1)
from sentence_transformers import SentenceTransformer
import torch

# model_id='/home/jh/semeval/sentence-transformers/paraphrase-multilingual-mpnet-base-v2/'
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 
model_name_or_path='out_models/all-mpnet-base-v2'#'/media/jh/新加卷/2025_1_16/sentence-transformers/all-mpnet-base-v2/' #models/gte_gnd 可以 
model = SentenceTransformer(model_name_or_path, model_kwargs={"torch_dtype": torch.float16}, trust_remote_code=True)#, 
# model=model.to('cuda')
# 把labels的一维张量转换为二维张量
batch_size=32
texts=[label['sentence2'] for label in labels]
texts=[texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
if len(texts)*batch_size<len(labels):
    texts.append([label['sentence2'] for label in labels[len(texts)*batch_size:]])
y_test=[]
for text in texts:
    with torch.no_grad():
        embeddings = model.encode(text, convert_to_tensor=True)
    item=embeddings.cpu()
    y_test.append(item)
print(embeddings)

y_t=[]
i=0
for y in y_test:
    for e in y:
        item={}
        item['Code']=labels[i]['Code']
        item['embedding']=e
        item['sentence2']=labels[i]['sentence2']
        y_t.append(item)
        i+=1
print(len(y_t),len(labels)) # 

def process_data(en_de,root,des_root,mode='m'):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'title' in d:
                tmp=d
                break
        item={}
        if type(tmp['title'])==list:
            item['title']=' '.join(tmp['title'])
        else:
            item['title']=tmp['title']
        if type(tmp['abstract'])==list:
            item['abstract']='  '.join(tmp['abstract'])
        else:
            item['abstract']=tmp['abstract']
        return item
    x_test=[]
    file_names=[]
    des_path=[]
    for dirs in os.listdir(f'{root}/'):
        if not os.path.exists(f'{root}/{dirs}/{en_de}'):
            continue
        for file in os.listdir(f'{root}/{dirs}/{en_de}'):
            # if os.path.exists(f'{des_root}/{dirs}/{en_de}/{file}'):
            #     continue
            item=get_train_data(f'{root}/{dirs}/{en_de}/{file}')
            text=item['title']+'.'+item['abstract']
            x_test.append(text)
            # 把file的后缀名去掉
            file=file.split('.')[0]+'.json'
            file_names.append(f'{des_root}/{dirs}/{en_de}/{file}')
            des_path.append(f'{des_root}/{dirs}/{en_de}')
    batch_size=768
    texts=[x_test[i:i+batch_size] for i in range(0, len(x_test), batch_size)]
    if len(texts)*batch_size<len(file_names):
        texts.append([file_names[len(texts)*batch_size:]])
    result=[]
    index=0
    index1=0
    
    for text in texts:
        with torch.no_grad():
            embeddings = model.encode(text, convert_to_tensor=True)
        y_list=[y['embedding'].tolist() for y in y_t]
        # y_tt=torch.tensor([y['embedding'].tolist() for y in y_t]).to('cuda')
        # print(embeddings.shape,y_tt.shape)
        similarities=[]
        embeddings=embeddings.to('cuda')
        for cc in range(0,len(y_list),batch_size):
            y_tt=torch.tensor(y_list[cc:cc+batch_size]).to('cuda')
            with torch.no_grad():
                similaritie = model.similarity(y_tt.to(torch.float16),embeddings.to(torch.float16)).to('cpu')
            # print(similaritie.shape)
            similarities.extend(similaritie.tolist())
            # print(torch.tensor(similarities).shape) #(n,768)
        if mode=='m2':
            m2_similarities.extend(torch.tensor(similarities).T.tolist())
            print(torch.tensor(m2_similarities).shape)
            continue
        elif mode=='m1':
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=500, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                sims=[]
                for i in t:
                    sims.append(similarities[index1][i])
                index1+=1
                top_k_2=torch.topk(torch.tensor(sims), k=50, dim=0).indices.T.cpu().tolist()
                code=[]
                for i in top_k_2:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        else:
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=50, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                code=[]
                for i in t:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        for res in result:
            # 如果不存在目录，则创建目录
            if not os.path.exists(des_path[index]):
                os.makedirs(des_path[index])
            # if os.path.exists(file_names[index]):
            #     index+=1
            #     continue
            with open(file_names[index],'w') as f:
                json.dump(res,f,ensure_ascii=False,indent=4)
            index+=1
        print('batch',index)
        result=[] #重置result
# 打印运行时间
import time
start_time = time.time()
root1='/media/jh/新加卷1'
root='/media/jh/新加卷1/2024_11_10/llms4subjects-main/shared-task-datasets/TIBKAT/all-subjects/data/dev/' #测试数据集目录
des_root='/media/jh/新加卷1/2024_12_04/2025_1/test2' #生成的数据的目标目录
# model_id='./models/mpnet_gnd_triplet'#'/media/jh/新加卷/2025_1_16/final_model'
# # with open('./tmp/core_labels_gte.pkl','rb') as f:
# #     y_t=pickle.load(f)
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 

# m2_similarities=[]
process_data('en',root,des_root,mode='m') #15904
# with open('./tmp/m2_en.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
# m2_similarities=[]
process_data('de',root,des_root,mode='m') #15904
# with open('./tmp/m2_de.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
end_time = time.time()
print('运行时间:', end_time - start_time, '秒')

tensor([[ 0.0033, -0.0886, -0.0190,  ...,  0.0074, -0.0579, -0.0142],
        [-0.0396, -0.0846, -0.0084,  ...,  0.0313, -0.0635, -0.0269],
        [-0.0291, -0.0442,  0.0145,  ...,  0.0172, -0.0733, -0.0210]],
       device='cuda:0', dtype=torch.float16)
204739 204739
batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6045
batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6144
batch 6912
batch 7621
运行时间: 850.9448997974396 秒


In [3]:
from sentence_transformers import SentenceTransformer
import os
import json
import pickle
import torch
import os
import json
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
root='/media/jh/新加卷1'
with open(f'{root}/2024_11_10/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json') as f:
    gnd_jsons=json.load(f)
labels=[]
for c in gnd_jsons:
    item1={}
    item1['sentence2']='Classification Name is '+c['Classification Name']+'. Name is '+c['Name']
    item1['Code']=c['Code']
    labels.append(item1)
from sentence_transformers import SentenceTransformer
import torch

# model_id='/home/jh/semeval/sentence-transformers/paraphrase-multilingual-mpnet-base-v2/'
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 
model_name_or_path='tmp/out_models/paraphrase-MiniLM-L6-v2'#'/media/jh/新加卷/2025_1_16/sentence-transformers/all-mpnet-base-v2/' #models/gte_gnd 可以 
model = SentenceTransformer(model_name_or_path, model_kwargs={"torch_dtype": torch.float16}, trust_remote_code=True)#, 
# model=model.to('cuda')
# 把labels的一维张量转换为二维张量
batch_size=64
texts=[label['sentence2'] for label in labels]
texts=[texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
if len(texts)*batch_size<len(labels):
    texts.append([label['sentence2'] for label in labels[len(texts)*batch_size:]])
y_test=[]
for text in texts:
    with torch.no_grad():
        embeddings = model.encode(text, convert_to_tensor=True)
    item=embeddings.cpu()
    y_test.append(item)
print(embeddings)

y_t=[]
i=0
for y in y_test:
    for e in y:
        item={}
        item['Code']=labels[i]['Code']
        item['embedding']=e
        item['sentence2']=labels[i]['sentence2']
        y_t.append(item)
        i+=1
print(len(y_t),len(labels)) # 

def process_data(en_de,root,des_root,mode='m'):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'title' in d:
                tmp=d
                break
        item={}
        if type(tmp['title'])==list:
            item['title']=' '.join(tmp['title'])
        else:
            item['title']=tmp['title']
        if type(tmp['abstract'])==list:
            item['abstract']='  '.join(tmp['abstract'])
        else:
            item['abstract']=tmp['abstract']
        return item
    x_test=[]
    file_names=[]
    des_path=[]
    for dirs in os.listdir(f'{root}/'):
        if not os.path.exists(f'{root}/{dirs}/{en_de}'):
            continue
        for file in os.listdir(f'{root}/{dirs}/{en_de}'):
            # if os.path.exists(f'{des_root}/{dirs}/{en_de}/{file}'):
            #     continue
            item=get_train_data(f'{root}/{dirs}/{en_de}/{file}')
            text=item['title']+'.'+item['abstract']
            x_test.append(text)
            # 把file的后缀名去掉
            file=file.split('.')[0]+'.json'
            file_names.append(f'{des_root}/{dirs}/{en_de}/{file}')
            des_path.append(f'{des_root}/{dirs}/{en_de}')
    print('x_test:',len(x_test))
    if en_de=='de':
        with open('tmp/x_train_de2en.json','r') as f:
            x_train=json.load(f)
        x_test=[x['sentence1'] for x in x_train if x['label']!=0]
        # 德语翻译成英语后进行测试。x_train_de2en.json存放的是all-subjects_dev_de.json经过翻译后的数据
        print('x_test替换成功!')
    print('x_test:',len(x_test))
    batch_size=768
    texts=[x_test[i:i+batch_size] for i in range(0, len(x_test), batch_size)]
    if len(texts)*batch_size<len(file_names):
        texts.append([file_names[len(texts)*batch_size:]])
    result=[]
    index=0
    index1=0
    
    for text in texts:
        with torch.no_grad():
            embeddings = model.encode(text, convert_to_tensor=True)
        y_list=[y['embedding'].tolist() for y in y_t]
        # y_tt=torch.tensor([y['embedding'].tolist() for y in y_t]).to('cuda')
        # print(embeddings.shape,y_tt.shape)
        similarities=[]
        embeddings=embeddings.to('cuda')
        for cc in range(0,len(y_list),batch_size):
            y_tt=torch.tensor(y_list[cc:cc+batch_size]).to('cuda')
            with torch.no_grad():
                similaritie = model.similarity(y_tt.to(torch.float16),embeddings.to(torch.float16)).to('cpu')
            # print(similaritie.shape)
            similarities.extend(similaritie.tolist())
            # print(torch.tensor(similarities).shape) #(n,768)
        if mode=='m2':
            m2_similarities.extend(torch.tensor(similarities).T.tolist())
            print(torch.tensor(m2_similarities).shape)
            continue
        elif mode=='m1':
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=500, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                sims=[]
                for i in t:
                    sims.append(similarities[index1][i])
                index1+=1
                top_k_2=torch.topk(torch.tensor(sims), k=50, dim=0).indices.T.cpu().tolist()
                code=[]
                for i in top_k_2:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        else:
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=50, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                code=[]
                for i in t:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
        for res in result:
            # 如果不存在目录，则创建目录
            if not os.path.exists(des_path[index]):
                os.makedirs(des_path[index])
            # if os.path.exists(file_names[index]):
            #     index+=1
            #     continue
            with open(file_names[index],'w') as f:
                json.dump(res,f,ensure_ascii=False,indent=4)
            index+=1
        print('batch',index)
        result=[] #重置result
# 打印运行时间
import time
start_time = time.time()
root1='/media/jh/新加卷1'
root='/media/jh/新加卷1/2024_11_10/llms4subjects-main/shared-task-datasets/TIBKAT/all-subjects/data/dev/'
des_root='/media/jh/新加卷1/2024_12_04/2025/test3-5'
# model_id='./models/mpnet_gnd_triplet'#'/media/jh/新加卷/2025_1_16/final_model'
# # with open('./tmp/core_labels_gte.pkl','rb') as f:
# #     y_t=pickle.load(f)
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 

# m2_similarities=[]
# process_data('en',root,des_root,mode='m') #15904
# with open('./tmp/m2_en.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
# m2_similarities=[]
process_data('de',root,des_root,mode='m') #15904
# with open('./tmp/m2_de.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
end_time = time.time()
print('运行时间:', end_time - start_time, '秒')

tensor([[-0.6372,  0.2573, -0.7280,  ..., -0.5933, -0.0065,  0.7397],
        [-0.6592,  0.2507, -0.7104,  ..., -0.5747,  0.0039,  0.7622],
        [-0.6904,  0.2585, -0.7319,  ..., -0.5820,  0.0142,  0.8120]],
       device='cuda:0', dtype=torch.float16)
204739 204739
x_test: 7621
x_test替换成功!
x_test: 7621
batch 768
batch 1536
batch 2304
batch 3072
batch 3840
batch 4608
batch 5376
batch 6144
batch 6912
batch 7621
运行时间: 320.1072027683258 秒
